# Merge the Mexican Federal Budget files

In [1]:
from sys import stdout

## Data ingestion

Ingest all columns as raw strings.

In [2]:
def read_columns(file):
    with open(file, encoding='iso-8859-1') as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [3]:
def force_strings(columns):
    for column in columns:
        yield column, str

In [15]:
import cchardet as chardet

def detect_encoding(years, file_format):
    for year in years:
        filename = file_format % year

        with open(filename, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        print('Inspecting', filename, result)

In [16]:
from pandas import read_csv

def load_csv_files(years, file_format, encoding_):
    df = {}
    
    for year in years:
        filename = file_format % year
        columns = read_columns(filename)
        types = dict(force_strings(columns))
        
        print('Loading', filename, 'with encoding:', encoding_)
        stdout.flush()
        
        df[year] = read_csv(filename, encoding=encoding_, dtype=types)
    
    return df

## Column mapping

In [17]:
from sys import stdout

def strip_blanks(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            if column.strip() != column:
                print(year, column, 'stripped blanks')
                stdout.flush()

In [18]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [19]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [20]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [21]:
def alias_columns(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS: ', column)
                    stdout.flush()
        print('\nDone mapping', year, '\n')
        stdout.flush()

In [22]:
from pandas import DataFrame

def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    return overview

## Parse amount columns

There's a little cleaning to do on the amount columns (zeros are represented by a dash and there is some whitespace padding). I might as well take care of this here to make sure I don't run into parsing problems later).

In [23]:
from numpy import nan

amounts = ['Ejercido', 'Devengado', 'Aprobado', 'Pagado', 'Modificado', 'Adefas', 'Ejercicio']

def clean_amount_columns(batch):
    for year, df in sorted(batch.items()):
        for amount in amounts:
            try:
                # Weird behaviour from pandas: although I coerced columns to strings, 
                # there are some nan values which pandas considers as floats.
                series = batch[year][amount].apply(lambda x: '' if x is nan else x)
                series = series.apply(lambda x: x.strip())
                # '-' seem to represent a zero, judging from the value counts
                series = series.apply(lambda x: '0' if x == '-' else x)
                series = series.apply(lambda x: x.replace(',', ''))
                batch[year][amount] = series
                print(year, amount, 'cleaned numerical column')
            except KeyError:
                pass

In [27]:
from pandas import concat

def do_pipeline(output_file):
    print('\nLoading files...\n')
    
    # Load Cuenta Publica files
    cp_years = range(2010, 2016)
    cp_file_format = 'Cuenta_Publica_%s.csv'
    detect_encoding(cp_years, cp_file_format)
    cp = load_csv_files(cp_years, cp_file_format, 'windows-1252')
    
    # Load PEF 2016
    pef_years = [2016]
    pef_file_format = 'PEF%s_AC01.csv'
    detect_encoding(pef_years, pef_file_format)
    pef = load_csv_files(pef_years, pef_file_format, 'cp850')
    
    # Assemble the two datasets in one dictionary
    datasets = cp
    datasets.update(pef)
    
    print('\nCleaning data...\n')
    strip_blanks(datasets)
    delete_empty_columns(datasets)
    
    print('\nMapping columns...\n')
    aliases = load_aliases('mexican_federal_budget_column_nomenclature.yaml')
    alias_columns(datasets, aliases)

    print('\nBuilding column overview...\n')
    overview_table = build_overview(datasets)
    
    print('\nCleaning numerical columns...\n')
    clean_amount_columns(datasets)
    
    print('\nMerging datasets...\n')
    merged_dataset = concat(list(datasets.values()))
    
    print('\nSaving to', output_file, '\n')    
    mexico.to_csv(output_file, encoding='utf-8', index=False)
    
    print('\nPipeline done!', '\n')
    return merged_dataset, overview_table

In [28]:
mexico, overview = do_pipeline('mexican_federal_budget.2010_to_2016.merged.csv')


Loading files...

Inspecting Cuenta_Publica_2010.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Inspecting Cuenta_Publica_2011.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Inspecting Cuenta_Publica_2012.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Inspecting Cuenta_Publica_2013.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Inspecting Cuenta_Publica_2014.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Inspecting Cuenta_Publica_2015.csv {'encoding': 'WINDOWS-1252', 'confidence': 0.9900000095367432}
Loading Cuenta_Publica_2010.csv with encoding: windows-1252
Loading Cuenta_Publica_2011.csv with encoding: windows-1252
Loading Cuenta_Publica_2012.csv with encoding: windows-1252
Loading Cuenta_Publica_2013.csv with encoding: windows-1252
Loading Cuenta_Publica_2014.csv with encoding: windows-1252
Loading Cuenta_Publica_2015.csv with encoding: windows-1252
Inspecting PEF2016_AC01.csv {'enc

## Quality control

In [29]:
mexico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1325512 entries, 0 to 239082
Data columns (total 35 columns):
Actividad Institucional                                    1325512 non-null object
Adefas                                                     482560 non-null object
Aprobado                                                   1325512 non-null object
Ciclo                                                      1325512 non-null object
Clave de cartera                                           1057671 non-null object
Descripción de Fuente de Financiamiento                    1325511 non-null object
Descripción de Función                                     1325512 non-null object
Descripción de Grupo Funcional                             1325512 non-null object
Descripción de Objeto del Gasto                            1325483 non-null object
Descripción de Programa Presupuestario                     1325511 non-null object
Descripción de Ramo                                        

In [30]:
mexico.sample(n=20)

,Actividad Institucional,Adefas,Aprobado,Ciclo,Clave de cartera,Descripción de Fuente de Financiamiento,Descripción de Función,Descripción de Grupo Funcional,Descripción de Objeto del Gasto,Descripción de Programa Presupuestario,...,Modalidad del Programa presupuestario,Modificado,Objeto del Gasto,Pagado,Programa Presupuestario,Ramo,Reasignacion,Subfunción,Tipo de Gasto,Unidad Responsable
84565,7,NaN,58434.0,2013,0,Recursos fiscales,Comunicaciones,Desarrollo Económico,Otros servicios comerciales,Servicios de correo,...,E,NaN,33602,NaN,12,9,NaN,1,1,J9E
127579,2,NaN,33461,2012,0,Recursos fiscales,Educación,Desarrollo Social,Funerales y pagas de defunción,Actividades de apoyo administrativo,...,M,NaN,39101,NaN,1,11,NaN,6,1,712
29850,2,NaN,121678.0,2010,NaN,Recursos fiscales,Comunicaciones y Transportes,Desarrollo Económico,Mantenimiento y conservación de inmuebles,Actividades de apoyo administrativo,...,M,NaN,3504,NaN,1,9,NaN,6,1,630
138361,5,0,4297,2014,0,Recursos fiscales,"Asuntos Económicos, Comerciales y Laborales en...",Desarrollo Económico,Material eléctrico y electrónico,Ejecución a nivel nacional de los programas y ...,...,E,10848,24601,10848,3,14,NaN,2,1,125
151305,3,NaN,0,2013,1216B000109,Recursos fiscales,"Agropecuaria, Silvicultura, Pesca y Caza",Desarrollo Económico,Información en medios masivos derivada de la o...,Estabilización de cuencas y acuíferos.,...,K,NaN,33605,NaN,141,16,NaN,5,3,B00
236209,1,NaN,11838,2012,0,Recursos fiscales,Coordinación de la Política de Gobierno,Gobierno,Refacciones y accesorios menores de equipo de ...,Capacitar y educar para el ejercicio democráti...,...,R,NaN,29601,NaN,3,22,NaN,6,1,300
31488,5,NaN,0,2011,NaN,Recursos fiscales,Comunicaciones y Transportes,Desarrollo Económico,"Servicios de lavandería, limpieza e higiene","Supervisión, inspección y verificación del tra...",...,G,NaN,35801,NaN,2,9,NaN,3,1,626
13180,6,NaN,550271,2012,0,Recursos fiscales,Coordinación de la Política de Gobierno,Gobierno,Servicio telefónico convencional,Conducción de la política interior y las relac...,...,P,NaN,31401,NaN,1,4,NaN,2,1,213
23435,23,NaN,45000,2012,0,Recursos fiscales,Justicia,Gobierno,Pasajes terrestres nacionales para labores en ...,Atención Integral a Familiares de Personas Des...,...,E,NaN,37201,NaN,34,6,NaN,4,1,AYJ
59734,10,NaN,713000,2011,NaN,Recursos fiscales,Educación,Desarrollo Social,Material de limpieza,Normar los servicios educativos,...,G,NaN,21601,NaN,1,11,NaN,8,1,311


In [31]:
overview

,Column,2010,2011,2012,2013,2014,2015,2016
0,Descripción de la modalidad del programa presu...,True,True,True,True,True,True,True
1,Descripción de Subfunción,True,True,True,True,True,True,True
2,Clave de cartera,False,False,True,True,True,True,True
3,Grupo Funcional,True,True,True,True,True,True,True
4,Actividad Institucional,True,True,True,True,True,True,True
5,Descripción de Fuente de Financiamiento,True,True,True,True,True,True,True
6,Devengado,False,False,False,True,True,True,False
7,Descripción de Unidad Responsable,True,True,True,True,True,True,True
8,Modificado,False,False,False,False,True,True,False
9,Ramo,True,True,True,True,True,True,True


In [32]:
ls -lh | grep mexican_federal_budget.2010_to_2016

-rw-rw-r-- 1 loic loic 610M Aug  9 17:58 mexican_federal_budget.2010_to_2016.merged.csv


In [33]:
cat mexican_federal_budget.2010_to_2016.merged.csv | head -n 10

Actividad Institucional,Adefas,Aprobado,Ciclo,Clave de cartera,Descripción de Fuente de Financiamiento,Descripción de Función,Descripción de Grupo Funcional,Descripción de Objeto del Gasto,Descripción de Programa Presupuestario,Descripción de Ramo,Descripción de Reasignacion,Descripción de Subfunción,Descripción de Tipo de Gasto,Descripción de Unidad Responsable,Descripción de la Actividad Institucional,Descripción de la entidad federativa,Descripción de la modalidad del programa presupuestario,Devengado,Ejercicio,Ejercido,Entidad Federativa,Fuente de Financiamiento,Función,Grupo Funcional,Modalidad del Programa presupuestario,Modificado,Objeto del Gasto,Pagado,Programa Presupuestario,Ramo,Reasignacion,Subfunción,Tipo de Gasto,Unidad Responsable
4,,99305000,2016,0,Recursos fiscales,Legislación,Gobierno,Obra pública en bienes propios,Mantenimiento de Infraestructura,Poder Legislativo,Otros,Legislación,Gasto de obra pública,H. Cámara de Diputados,Llevar a cabo el proceso Legislativo,Dist